In [ ]:
import pyspark 

In [ ]:
conf = pyspark.SparkConf()
conf.setMaster('spark://spark-master:7077')
spark_context = pyspark.SparkContext(conf=conf) 

In [ ]:
# Открытие текстового файла и превращение его в RDD.
# К докеру со spark-worker примаунчена папка /user-data с вашего диска, в которой лежат скачанные данные.
data = spark_context.textFile('/user-data/combined/reviews.csv')
# Берём первую строку, для дальнейшей фильтрации. Первая строка отвечает за заголовки, они нас не интересуют.
header_data = data.first()
# Начались вычисления. Первая функция пропускает все данные кроме первой строки с заголовками.
# Функция `map` разделяет строку на колонки, используя разделитель `,`.
result_data = data.filter(lambda row: row != header_data) \
    .map(lambda line: line.split(","))

In [ ]:
result_data.take(10)

In [ ]:
result_data = result_data.filter(lambda line: len(line) > 5) 

In [ ]:
# Всего
print(result_data.count())
# Ненужных данных
print(result_data.filter(lambda line: not line[4].isdigit()).count()) 

In [ ]:
# Пропустим неправильно разбитые строки,
result_data = result_data.filter(lambda line: line[4].isdigit()) 

In [ ]:
# Нас интересуют всего два столбца — key и stars
result_data = result_data.map(lambda line: (line[1], int(line[4])))

In [ ]:
# Узнаем количество отзывов (каждому элементу добавляем 1)
result_data = result_data.map(lambda line: (line[0], (line[1], 1)))

In [ ]:
# Посчитаем сумму рейтингов и количества отзывов по ключу
result_data = result_data.reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1]))

In [ ]:
# посчитаем среднее арифметическое для каждой строки
result_data = result_data.mapValues(lambda x: x[0] / x[1]) 

In [ ]:
# Весь пайплайн выше можно переписать в одну команду
data = spark_context.textFile('/user-data/combined/reviews.csv')
header_data = data.first()
result_data = data.filter(lambda row: row != header_data) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line) > 5) \
    .filter(lambda line: line[4].isdigit()) \
    .map(lambda line: (line[1], int(line[4]))) \
    .map(lambda line: (line[0], (line[1], 1))) \
    .reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1])) \
    .mapValues(lambda x: x[0] / x[1]) 

In [ ]:
# сохраним данные в файл
result_data.coalesce(1).saveAsTextFile('/user-data/combined/results')